In [64]:
import numpy as np
import flickr_api
from flickr_api import Photo
from IPython.display import Image

from dataset.load import LoadDataset
from dataset.utils import load_credentials, get_photo

api_key, api_secret = load_credentials()
flickr_api.set_keys(api_key=api_key, api_secret=api_secret)

In [48]:
# Load h5 dataset
path = "data/paris_10000.h5"
dataset = LoadDataset(path)

# Print number of photos
print(dataset.size())

# Get first photo from it
it = dataset.get()
metadata, embedding = next(it)

# Load photo using its id, download and display it
p = Photo(id=metadata["id"])
img = get_photo(p)
display(img)

4216


ConnectionError: HTTPSConnectionPool(host='api.flickr.com', port=443): Max retries exceeded with url: /services/rest/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1206cd110>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [46]:
metadata, embedding = next(it)
metadata["id"]

'11462654004'

In [47]:
Photo(id=metadata["id"]).getAllContexts()

ConnectionError: HTTPSConnectionPool(host='api.flickr.com', port=443): Max retries exceeded with url: /services/rest/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x12082f9d0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [33]:
import pandas as pd
# http://librdf.org/flickcurl/api/flickcurl-searching-search-extras.html
info = p.getInfo(extras="count_faves")
pd.DataFrame(list(info.keys()))

,0
0,id
1,secret
2,server
3,farm
4,dateuploaded
5,isfavorite
6,license
7,safety_level
8,rotation
9,originalsecret


In [30]:
info["urls"]

{'url': [{'type': 'photopage',
   'text': 'https://www.flickr.com/photos/zedthedragon/10496686616/'}]}

In [49]:
# Try add location, groups (getAllContexts)
features = ["title", "people", "description", "comments", "views", "tags", "count_faves"]
x = {}
for feature in features:
    x[feature] = info[feature]
x

{'title': 'A sit in Paris :)',
 'people': {'haspeople': 0},
 'description': '<i>[<a href="http://www.flickr.com/photos/zedthedragon/sets/72157627672411104/with/6372195821/">My GETTY Images @</a>]</i> <i>[<a href="http://flickriver.com/photos/zedthedragon/popular-interesting/" rel="noreferrer nofollow">My MOST FAVE on Flickriver</a>]</i> <i>[<a href="http://www.fluidr.com/photos/tags/zedthedragon" rel="noreferrer nofollow">My RECENT on Fluidr</a>]</i> <i>[<a href="http://darckr.com/username?p=UEhQU0VTU0lEPWVkNTUxOWU2ODdmM2Q2ZmM4ODczMjlmYzAwOTNmNWNiJnVzZXJuYW1lPXplZCt0aGUrZHJhZ29u" rel="noreferrer nofollow">My STREAM on Darckr</a>]</i>\n\nA850 + 16-35 &amp; 2 photos\n',
 'comments': '169',
 'views': '33646',
 'tags': [Tag(id=b'49016227-104966866..., text=b'variosonnart281635...),
  Tag(id=b'49016227-104966866..., text=b'paris'),
  Tag(id=b'49016227-104966866..., text=b'zedthedragon'),
  Tag(id=b'49016227-104966866..., text=b'hdr'),
  Tag(id=b'49016227-104966866..., text=b'musee'),
  Tag(

In [70]:
path = "data/paris_10000.h5"
dataset = LoadDataset(path)
print(dataset.size())
it = dataset.get()

X = []
y = []
for metadata, embedding in it:
    X.append(embedding)
    y.append(int(metadata["views"]))

X = np.stack(X, axis=0)
y = np.asarray(y).astype(np.float32)
X.shape, y.shape

4216


((4216, 512), (4216,))

In [103]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = rf.predict(X_test)
r2_score(y_pred, y_test)

-0.11376805582615557

In [82]:
import torch
from torch import nn
import torch.nn.functional as F

class ScoreModel(nn.Module):
    def __init__(self, input_size):
        super(ScoreModel, self).__init__()
        self.linear1 = nn.Linear(input_size, 32)
        self.linear2 = nn.Linear(32, 16)
        self.linear3 = nn.Linear(16, 1)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        y = F.relu(self.linear3(x))
        return y

input_size = 512
model = ScoreModel(input_size)
X_train = torch.tensor(X)
y_train = torch.tensor(y)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for epoch in range(500):
    # Forward pass
    y_pred = model(X_train)

    # Loss
    loss = criterion(y_pred, y_train)
    if epoch % 100 == 99:
        print(epoch, loss.item())

    # Backprop and weight update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print("done.")

99 61705916.0
199 61705916.0
299 61705916.0
399 61705916.0
499 61705916.0
done.
